<a href="https://colab.research.google.com/github/DilemmaFixer3/AI_pr_5-6-7/blob/main/pr_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# dcgan_mnist.py
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import os

# --- Параметри ---
BUFFER_SIZE = 60000
BATCH_SIZE = 256
EPOCHS = 50
NOISE_DIM = 100
NUM_EXAMPLES_TO_GENERATE = 16
OUTPUT_DIR = "dcgan_output"

os.makedirs(OUTPUT_DIR, exist_ok=True)

# --- Підготовка даних (MNIST) ---
(train_images, _), (_, _) = tf.keras.datasets.mnist.load_data()
train_images = (train_images.astype('float32') - 127.5) / 127.5  # нормалізувати в [-1,1]
train_images = np.expand_dims(train_images, axis=-1)
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

# --- Побудова генератора ---
def make_generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(7*7*256, use_bias=False, input_shape=(NOISE_DIM,)))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())

    model.add(layers.Reshape((7, 7, 256)))  # (7,7,256)
    model.add(layers.Conv2DTranspose(128, (5,5), strides=(1,1), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())

    model.add(layers.Conv2DTranspose(64, (5,5), strides=(2,2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())

    model.add(layers.Conv2DTranspose(1, (5,5), strides=(2,2), padding='same', use_bias=False, activation='tanh'))
    # вихід: (28,28,1)
    return model

# --- Побудова дискримінатора ---
def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(64, (5,5), strides=(2,2), padding='same',
                             input_shape=[28,28,1]))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, (5,5), strides=(2,2), padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))
    return model

generator = make_generator_model()
discriminator = make_discriminator_model()

# --- Втрати і оптимайзери ---
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    return real_loss + fake_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

# --- Контрольні вектори для відображення прогресу ---
seed = tf.random.normal([NUM_EXAMPLES_TO_GENERATE, NOISE_DIM])

# --- Тренувальна крок-функція ---
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, NOISE_DIM])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    return gen_loss, disc_loss

# --- Функція для малювання і збереження зразків ---
def generate_and_save_images(model, epoch, test_input):
    predictions = model(test_input, training=False)
    fig = plt.figure(figsize=(4,4))

    for i in range(predictions.shape[0]):
        plt.subplot(4,4,i+1)
        plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
        plt.axis('off')

    plt.suptitle(f'Epoch {epoch}')
    fname = os.path.join(OUTPUT_DIR, f'image_at_epoch_{epoch:03d}.png')
    plt.savefig(fname)
    plt.close(fig)

# --- Основний цикл навчання ---
def train(dataset, epochs):
    for epoch in range(1, epochs+1):
        gen_loss_avg = 0.0
        disc_loss_avg = 0.0
        steps = 0
        for image_batch in dataset:
            # якщо на останньому батчі менше BATCH_SIZE, пропускаємо або падимо; можна зробити padded.
            if image_batch.shape[0] != BATCH_SIZE:
                continue
            g_loss, d_loss = train_step(image_batch)
            gen_loss_avg += g_loss
            disc_loss_avg += d_loss
            steps += 1
        if steps > 0:
            gen_loss_avg /= steps
            disc_loss_avg /= steps
        print(f'Epoch {epoch}, Gen loss: {gen_loss_avg:.4f}, Disc loss: {disc_loss_avg:.4f}')
        generate_and_save_images(generator, epoch, seed)

if __name__ == "__main__":
    train(train_dataset, EPOCHS)
    # зберегти моделі
    generator.save(os.path.join(OUTPUT_DIR, "generator.h5"))
    discriminator.save(os.path.join(OUTPUT_DIR, "discriminator.h5"))
    print("Навчання завершено. Зображення збережено в", OUTPUT_DIR)


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Epoch 1, Gen loss: 0.6998, Disc loss: 1.0981
Epoch 2, Gen loss: 1.1511, Disc loss: 0.9630
Epoch 3, Gen loss: 1.4080, Disc loss: 0.7271
Epoch 4, Gen loss: 1.4404, Disc loss: 0.9058
Epoch 5, Gen loss: 1.3225, Disc loss: 0.9907
Epoch 6, Gen loss: 1.0478, Disc loss: 1.2437
Epoch 7, Gen loss: 0.9248, Disc loss: 1.2143


KeyboardInterrupt: 

In [ ]:
# dcgan_mnist_fast.py
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import os

# --- Параметри (Зміни для прискорення) ---
# Зменшення епох для швидшого завершення
EPOCHS = 10
# Можна зменшити розмір батчу, щоб пришвидшити обробку одного кроку, але це може збільшити загальний час через більшу кількість кроків.
# Залишимо 256, оскільки зменшення EPOCHS - головний фактор.
BATCH_SIZE = 256
BUFFER_SIZE = 60000
NOISE_DIM = 100
NUM_EXAMPLES_TO_GENERATE = 16
OUTPUT_DIR = "dcgan_output_fast"

os.makedirs(OUTPUT_DIR, exist_ok=True)

# --- Підготовка даних (MNIST) ---
(train_images, _), (_, _) = tf.keras.datasets.mnist.load_data()
# Можна використати лише частину даних для дуже швидкого тесту (приміром, перші 10000)
# train_images = train_images[:10000] # Активація цієї лінії значно прискорить процес, але знизить якість
train_images = (train_images.astype('float32') - 127.5) / 127.5  # нормалізувати в [-1,1]
train_images = np.expand_dims(train_images, axis=-1)
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

# --- Побудова генератора (Без змін) ---
def make_generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(7*7*256, use_bias=False, input_shape=(NOISE_DIM,)))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())

    model.add(layers.Reshape((7, 7, 256)))  # (7,7,256)
    model.add(layers.Conv2DTranspose(128, (5,5), strides=(1,1), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())

    model.add(layers.Conv2DTranspose(64, (5,5), strides=(2,2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())

    model.add(layers.Conv2DTranspose(1, (5,5), strides=(2,2), padding='same', use_bias=False, activation='tanh'))
    # вихід: (28,28,1)
    return model

# --- Побудова дискримінатора (Без змін) ---
def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(64, (5,5), strides=(2,2), padding='same',
                            input_shape=[28,28,1]))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, (5,5), strides=(2,2), padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))
    return model

generator = make_generator_model()
discriminator = make_discriminator_model()

# --- Втрати і оптимайзери (Без змін) ---
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    return real_loss + fake_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

# Зменшення швидкості навчання (1e-4) є стандартним для стабілізації GAN, не змінюємо її для прискорення.
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

# --- Контрольні вектори для відображення прогресу (Без змін) ---
seed = tf.random.normal([NUM_EXAMPLES_TO_GENERATE, NOISE_DIM])

# --- Тренувальна крок-функція (Без змін) ---
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, NOISE_DIM])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    return gen_loss, disc_loss

# --- Функція для малювання і збереження зразків (Без змін) ---
def generate_and_save_images(model, epoch, test_input):
    predictions = model(test_input, training=False)
    fig = plt.figure(figsize=(4,4))

    for i in range(predictions.shape[0]):
        plt.subplot(4,4,i+1)
        # Масштабування назад у [0, 255]
        plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
        plt.axis('off')

    plt.suptitle(f'Epoch {epoch}')
    fname = os.path.join(OUTPUT_DIR, f'image_at_epoch_{epoch:03d}.png')
    plt.savefig(fname)
    plt.close(fig)

# --- Основний цикл навчання (Без змін, окрім OUTPUT_DIR) ---
def train(dataset, epochs):
    print(f"Початок навчання на {epochs} епохах...")
    for epoch in range(1, epochs+1):
        gen_loss_avg = 0.0
        disc_loss_avg = 0.0
        steps = 0
        for image_batch in dataset:
            # Вирішуємо проблему з неповним батчем, що може виникати наприкінці
            if image_batch.shape[0] != BATCH_SIZE:
                continue
            g_loss, d_loss = train_step(image_batch)
            gen_loss_avg += g_loss
            disc_loss_avg += d_loss
            steps += 1

        if steps > 0:
            gen_loss_avg /= steps
            disc_loss_avg /= steps
        print(f'Epoch {epoch}, Gen loss: {gen_loss_avg:.4f}, Disc loss: {disc_loss_avg:.4f}')
        # Зображення зберігаються на кожній епосі
        generate_and_save_images(generator, epoch, seed)

if __name__ == "__main__":
    train(train_dataset, EPOCHS)
    # зберегти моделі
    generator.save(os.path.join(OUTPUT_DIR, "generator.h5"))
    discriminator.save(os.path.join(OUTPUT_DIR, "discriminator.h5"))
    print("Навчання завершено. Зображення збережено в", OUTPUT_DIR)

Початок навчання на 10 епохах...
Epoch 1, Gen loss: 0.7807, Disc loss: 1.0174
Epoch 2, Gen loss: 1.2566, Disc loss: 0.7945
Epoch 3, Gen loss: 1.9319, Disc loss: 0.5209
Epoch 4, Gen loss: 1.6638, Disc loss: 0.7471
Epoch 5, Gen loss: 1.4746, Disc loss: 0.8608
Epoch 6, Gen loss: 1.2361, Disc loss: 1.1015
Epoch 7, Gen loss: 1.0337, Disc loss: 1.1159
Epoch 8, Gen loss: 1.1031, Disc loss: 1.0916
Epoch 9, Gen loss: 1.2178, Disc loss: 1.0447
Epoch 10, Gen loss: 1.1411, Disc loss: 1.0780


Навчання завершено. Зображення збережено в dcgan_output_fast
